In [1]:
import os
import sys
# from langchain_community.document_loaders import pypdf
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Weaviate
from langchain.agents import AgentType, initialize_agent
from langchain.schema import SystemMessage
from langchain.embeddings import OpenAIEmbeddings
from typing import List,Union
from weaviate.embedded import EmbeddedOptions
from dotenv import load_dotenv

import sys
sys.path.append('../utils')  # Adjust the path accordingly



In [2]:
from splitters import *
from retrivers import *

Choosing an appropriate chunking and text splitter mechanism is crutial for the performance of the RAG system

In [3]:
from langchain_community.document_loaders import PyPDFLoader
# Lets start by loading the document 
loader = PyPDFLoader('../data/RobinsonAdvisory.pdf')
contract = loader.load()

In [ ]:
%load_ext autoreload
%autoreload 2


In [4]:
recursive_doc = RecursiveSplitter(contract)

In [7]:
character_doc = characterSplitter(contract)

In [8]:
semantic_doc = SemanticSplitter(contract)

/home/kerod/Desktop/week_11/RAG-system-for-Contract-Q-A/week_11/lib/python3.10/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


In [9]:
qa_docx_file_path = "../data/Robinson Q&A.docx"


In [11]:
from langchain.vectorstores import Weaviate
import docx
from langchain_openai import ChatOpenAI

import weaviate
from dotenv import load_dotenv,find_dotenv
from weaviate.embedded import EmbeddedOptions

In [20]:
def create_vectorStore(doc):
    client = weaviate.Client(embedded_options=EmbeddedOptions)
    vectorstore = Weaviate.from_documents(client =  client,
                                        documents= doc,
                                        embedding=OpenAIEmbeddings(),
                                        by_text = False)
    return vectorstore

In [21]:
chara_vS = create_vectorStore(character_doc)
recur_vs = create_vectorStore(recursive_doc)
sem_vs = create_vectorStore(semantic_doc)

/home/kerod/Desktop/week_11/RAG-system-for-Contract-Q-A/week_11/lib/python3.10/site-packages/weaviate/warnings.py:158: DeprecationWarning: Dep016: You are using the Weaviate v3 client, which is deprecated.
            Consider upgrading to the new and improved v4 client instead!
            See here for usage: https://weaviate.io/developers/weaviate/client-libraries/python
            
  warnings.warn(


embedded weaviate is already listening on port 8079


/home/kerod/Desktop/week_11/RAG-system-for-Contract-Q-A/week_11/lib/python3.10/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


embedded weaviate is already listening on port 8079
embedded weaviate is already listening on port 8079


In [22]:
char_retriver = chara_vS.as_retriever()
recur_retriver = recur_vs.as_retriever()
sem_retriver = sem_vs.as_retriever()


In [23]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.runnables import (
    RunnableParallel,
    RunnablePassthrough
)
from langchain.schema.output_parser import StrOutputParser

In [26]:
def llm_chain(retriever):
    # Define LLM
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

    # Define prompt template
    prompt_template = """You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the question. 
    If you don't know the answer, just say that you don't know. 
    Question: {question} 
    Context: {context} 
    Answer:
    """

    prompt = ChatPromptTemplate.from_template(prompt_template)


    retrieval = RunnableParallel(
        {"context": retriever,  "question": RunnablePassthrough()} 
    )

    chain = retrieval | prompt | llm | StrOutputParser()

    return chain

In [27]:
char_chain = llm_chain(char_retriver)
recur_chain = llm_chain(recur_retriver)
sem_chain = llm_chain(sem_retriver)

In [29]:
from docx import Document
def extract_questions_answers(docx_file):
    questions = []
    answers = []

    doc = Document(docx_file)

    # Iterate through paragraphs
    for i in range(len(doc.paragraphs)):
        text = doc.paragraphs[i].text.strip()
        # Alternate paragraphs are questions and answers
        if text.startswith("Q"):
            question = text.split(": ", 1)[-1].strip()
            questions.append(question)
        elif text.startswith("A"):
            answer = text.split(": ", 1)[-1].strip()
            answers.append(answer)
            
            i += 1
        else:
            i += 1
        
        # if i % 2 == 0:
        #     questions.append(doc.paragraphs[i].text)
        # else:
        #     answers.append(doc.paragraphs[i].text)

    return questions, answers

questions, ground_truth = extract_questions_answers(qa_docx_file_path)


In [33]:
def ragas(retriever, chain, question):
    contexts = []
    answers = []
    for question in questions:
        contexts.append([docs.page_content for docs in retriever.get_relevant_documents(question)])
        answers.append(chain.invoke(question))

    return contexts, answers

In [45]:
ground_truths = [[x] for x in ground_truth]

In [36]:
char_contex, char_answer = ragas(char_retriver, char_chain, questions)
rec_context, rec_answer = ragas(recur_retriver, recur_chain, questions)
sem_contex, sec_answer= ragas(sem_retriver, sem_chain, questions)


/home/kerod/Desktop/week_11/RAG-system-for-Contract-Q-A/week_11/lib/python3.10/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/home/kerod/Desktop/week_11/RAG-system-for-Contract-Q-A/week_11/lib/python3.10/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/home/kerod/Desktop/week_11/RAG-system-for-Contract-Q-A/week_11/lib/python3.10/site-packages/pydantic/main.py:

In [37]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

/home/kerod/Desktop/week_11/RAG-system-for-Contract-Q-A/week_11/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [46]:
def evaluation (question, answers, contexts, ground_truths):
    # Preparing the dataset
    data = {
                "question": questions,
                "answer": answers,
                "contexts": contexts,
                "ground_truth": ground_truths
            }

    # Convert dict to dataset
    dataset = Dataset.from_dict(data)

    result = evaluate(
        dataset=dataset, 
        metrics=[
            context_precision,
            context_recall,
            faithfulness,
            answer_relevancy,
        ],
    )

    df = result.to_pandas()
        
    return df

In [47]:
char_evaluation = evaluation(questions, char_answer, char_contex, ground_truth)

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]/home/kerod/Desktop/week_11/RAG-system-for-Contract-Q-A/week_11/lib/python3.10/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/home/kerod/Desktop/week_11/RAG-system-for-Contract-Q-A/week_11/lib/python3.10/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/home/kerod/Desktop/week_11/RAG-system-for-Contract-Q-A/week

In [55]:
rec_evaluation = evaluation(questions, rec_answer, rec_context, ground_truth)

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]/home/kerod/Desktop/week_11/RAG-system-for-Contract-Q-A/week_11/lib/python3.10/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/home/kerod/Desktop/week_11/RAG-system-for-Contract-Q-A/week_11/lib/python3.10/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/home/kerod/Desktop/week_11/RAG-system-for-Contract-Q-A/week

ExceptionInRunner: The runner thread which was running the jobs raised an exeception. Read the traceback above to debug it. You can also pass `raise_exception=False` incase you want to show only a warning message instead.

In [49]:
sem_evaluation = evaluation(questions, sec_answer, sem_contex, ground_truth)

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]/home/kerod/Desktop/week_11/RAG-system-for-Contract-Q-A/week_11/lib/python3.10/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/home/kerod/Desktop/week_11/RAG-system-for-Contract-Q-A/week_11/lib/python3.10/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/home/kerod/Desktop/week_11/RAG-system-for-Contract-Q-A/week

In [51]:
char_evaluation

,question,answer,contexts,ground_truth,context_precision,context_recall,faithfulness,answer_relevancy
0,Who are the parties to the Agreement and what ...,The parties to the Agreement are Cloud Investm...,[ADVISOR Y\nSER VICES\nAGREEMENT\nThis\nAdviso...,Cloud Investments Ltd. (“Company”) and Jack Ro...,1.000000,0.5,1.000000,0.850181
1,What is the termination notice?,The termination notice allows either party to ...,[-\n2\n-\n4.\nTerm\n:\nThe\nterm\nof\nthis\nAg...,According to section 4:14 days for convenience...,1.000000,1.0,1.000000,0.935779
2,What are the payments to the Advisor under the...,The payments to the Advisor under the Agreemen...,[-\n2\n-\n4.\nTerm\n:\nThe\nterm\nof\nthis\nAg...,According to section 6: 1. Fees of $9 per hour...,1.000000,1.0,1.000000,1.000000
3,Can the Agreement or any of its obligations be...,"No, the Agreement cannot be assigned, sold, de...",[ADVISOR Y\nSER VICES\nAGREEMENT\nThis\nAdviso...,1. Under section 1.1 the Advisor can’t assign ...,1.000000,1.0,1.000000,0.928336
4,Who owns the IP?,The IP is fully and exclusively owned by the C...,"[-\n4\n-\nConfidentiality,\nNone\nCompete\nand...",According to section 4 of the Undertaking (App...,1.000000,1.0,1.000000,0.888741
5,Is there a non-compete obligation to the Advisor?,"Yes, there is a non-compete obligation to the ...",[ADVISOR Y\nSER VICES\nAGREEMENT\nThis\nAdviso...,Yes. During the term of engagement with the Co...,0.805556,1.0,1.000000,0.973648
6,Can the Advisor charge for meal time?,"No, the Advisor cannot charge for meal time as...",[-\n2\n-\n4.\nTerm\n:\nThe\nterm\nof\nthis\nAg...,"No. See Section 6.1, Billable Hour doesn’t inc...",1.000000,0.0,NaN,0.742687
7,In which street does the Advisor live?,"The Advisor lives at 1 Rabin Street, Tel Aviv,...",[ADVISOR Y\nSER VICES\nAGREEMENT\nThis\nAdviso...,"1 Rabin st, Tel Aviv, Israel",1.000000,1.0,1.000000,0.954076
8,Is the Advisor entitled to social benefits?,"No, the Advisor is not entitled to social bene...",[ADVISOR Y\nSER VICES\nAGREEMENT\nThis\nAdviso...,"No. According to section 8 of the Agreement, t...",1.000000,0.0,0.666667,0.939500
9,What happens if the Advisor claims compensatio...,If the Advisor claims compensation based on em...,[-\n2\n-\n4.\nTerm\n:\nThe\nterm\nof\nthis\nAg...,If the Advisor is determined to be an employee...,0.583333,1.0,1.000000,0.785026


In [52]:
sem_evaluation

,question,answer,contexts,ground_truth,context_precision,context_recall,faithfulness,answer_relevancy
0,Who are the parties to the Agreement and what ...,The parties to the Agreement are Cloud Investm...,[Entir e\nAgreement;\nNo\nWaiver\nor\nAssignme...,Cloud Investments Ltd. (“Company”) and Jack Ro...,1.000000,1.0,NaN,0.953996
1,What is the termination notice?,The termination notice is a written notice tha...,[-\n2\n-\n4. Term\n:\nThe\nterm\nof\nthis\nAgr...,According to section 4:14 days for convenience...,1.000000,0.5,1.0,0.957800
2,What are the payments to the Advisor under the...,The payments to the Advisor under the Agreemen...,"[-\n3\n-\nconstitute\nsalary\npayments,\nand\n...",According to section 6: 1. Fees of $9 per hour...,0.500000,1.0,1.0,1.000000
3,Can the Agreement or any of its obligations be...,"No, the Agreement or any of its obligations ca...",[Entir e\nAgreement;\nNo\nWaiver\nor\nAssignme...,1. Under section 1.1 the Advisor can’t assign ...,1.000000,1.0,1.0,0.000000
4,Who owns the IP?,The IP is fully and exclusively owned by the C...,"[-\n4\n-\nConfidentiality,\nNone\nCompete\nand...",According to section 4 of the Undertaking (App...,0.833333,1.0,1.0,0.957935
5,Is there a non-compete obligation to the Advisor?,"Yes, there is a non-compete obligation to the ...",[NOW\nTHEREFORE\nTHE\nPARTIES\nAGREE\nAS\nFOLL...,Yes. During the term of engagement with the Co...,1.000000,1.0,0.5,0.999732
6,Can the Advisor charge for meal time?,"No, the Advisor cannot charge for meal time as...",[Advisor\nshall\nnot\nreceive\nnor\nbe\nentitl...,"No. See Section 6.1, Billable Hour doesn’t inc...",0.638889,0.0,1.0,1.000000
7,In which street does the Advisor live?,"The Advisor lives at 1 Rabin Street, Tel Aviv,...",[ADVISOR Y\nSER VICES\nAGREEMENT\nThis\nAdviso...,"1 Rabin st, Tel Aviv, Israel",1.000000,1.0,1.0,0.954076
8,Is the Advisor entitled to social benefits?,"No, the Advisor is not entitled to social bene...",[Advisor\nshall\nnot\nreceive\nnor\nbe\nentitl...,"No. According to section 8 of the Agreement, t...",1.000000,0.0,1.0,0.970786
9,What happens if the Advisor claims compensatio...,If the Advisor claims compensation based on an...,"[-\n3\n-\nconstitute\nsalary\npayments,\nand\n...",If the Advisor is determined to be an employee...,0.916667,1.0,1.0,0.914045


In [56]:
char_context_precision = char_evaluation['context_precision'].mean()
char_recall = char_evaluation['context_recall'].mean()
char_faithfulness = char_evaluation['faithfulness'].mean()
char_answer_relevancy = char_evaluation['answer_relevancy'].mean()

sem_context_precision = sem_evaluation['context_precision'].mean()
sem_recall = sem_evaluation['context_recall'].mean()
sem_faithfulness = sem_evaluation['faithfulness'].mean()
sem_answer_relevancy = sem_evaluation['answer_relevancy'].mean()



In [57]:
print(f"The average score of context precison of char splitter is {char_context_precision}")
print(f"The average score of context recall of char splitter is {char_recall}")
print(f"The average score of faithfulness of char splitter is {char_faithfulness}")
print(f"The average score of answer relevancy of char splitter is {char_answer_relevancy}")

print(f"The average score of context precison of recursive splitter is {sem_context_precision}")
print(f"The average score of context recall of recursive splitter is {sem_recall}")
print(f"The average score of faithfulness of recursive splitter is {sem_faithfulness}")
print(f"The average score of answer relevancy of recursive splitter is {sem_answer_relevancy}")


The average score of context precison of char splitter is 0.9388888888407869
The average score of context recall of char splitter is 0.75
The average score of faithfulness of char splitter is 0.9629629629629631
The average score of answer relevancy of char splitter is 0.8997974259885055
The average score of context precison of recursive splitter is 0.888888888844537
The average score of context recall of recursive splitter is 0.75
The average score of faithfulness of recursive splitter is 0.9444444444444444
The average score of answer relevancy of recursive splitter is 0.8708370359640367
